In [63]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [64]:
#import the libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from operator import itemgetter
from keras.models import Model
from keras.layers import Input, LSTM, Dense


In [65]:
class Vocabulary:
  def __init__(self, name):
    self.name = name
    self.token2index = {"PAD":0 , "SOS":1 , "EOS":2}
    self.token2count = {}
    self.index2token = {0: "PAD", 1: "SOS", 2: "EOS"}
    self.num_tokens = 4
    self.num_token_line=0
    self.source_line=[]
    self.target_line=[]
    self.max_source_line_len=0
    self.max_target_line_len=0

  def add_token(self, token):
    if token not in self.token2index:
      self.token2index[token] = self.num_tokens
      self.token2count[token] = 1
      self.index2token[self.num_tokens] = token
      self.num_tokens += 1
    else:
      self.token2count[token] += 1
            
  def build_vocabulary(self, sourceLineTokens,targetLineTokens):        
      np.array(sourceLineTokens).reshape(-1,1)
      np.array(targetLineTokens).reshape(-1,1)

      for i in sourceLineTokens:
        self.source_line.append(i)
        s=eval(i) 
        if(self.max_source_line_len < len(s)):
          self.max_source_line_len=len(s)         
        for j in s:
          self.add_token(j)
      for i in targetLineTokens:
        self.target_line.append(i)
        s=eval(i)
        if(self.max_target_line_len < len(s)):
          self.max_target_line_len=len(s)           
        for j in s:
          self.add_token(j)
      
      return  self.token2index, self.token2count, self.index2token, self.num_tokens, self.source_line, self.target_line

  def get_top_k_token(self):
    top_k_token_count=dict(sorted(self.token2count.items(), key = itemgetter(1), reverse = True)[:top_k])
    top_k_token2index = {"PAD":0, "SOS":1, "EOS":2, "OOV_TOKEN":3}
    top_k_index2token = {0: "PAD", 1: "SOS", 2: "EOS", 3: "OOV_TOKEN"}
    num_count = 4
    for i in top_k_token_count:
      top_k_token2index[i] = num_count
      top_k_index2token[num_count] = i
      num_count += 1

    return top_k_token_count, top_k_token2index, top_k_index2token
  

In [66]:
#import the data sheet
data=pd.read_csv('/content/gdrive/MyDrive/train.csv')
top_k=1000
latent_dim=512
batch_size=64
epochs = 60
maxlen=30

#Build the vocabulary
vocab=Vocabulary("vocabulary")
token2index,token2count,index2token,num_tokens,source_line,target_line=vocab.build_vocabulary(data['sourceLineTokens'] , data['targetLineTokens'] )

token2count,token2index,index2token=vocab.get_top_k_token()
print("Vocabulary Created")

Vocabulary Created


In [67]:
def convert_to_index_seq(token_line):
  token_index_seq=[]
  for i in token_line:
    tokens=eval(i)
    temp_list=[]     
    for tok in tokens:      
      if tok in token2index.keys():
        temp_list.append(token2index[tok])
      else:
        temp_list.append(3)
    token_index_seq.append(np.array([1]+temp_list+[2]))
  return np.array(token_index_seq)


**Convert to index Sequence (Training Set)**

In [68]:
token_index_sourceline=convert_to_index_seq(source_line)
token_index_targetline=convert_to_index_seq(target_line)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


**PAD The Token index sequence (Trainning Set)**

In [69]:
#pad the token_index_seq (Training Set)
padded_source_line = tf.keras.preprocessing.sequence.pad_sequences(token_index_sourceline,padding='post', maxlen = maxlen)
padded_target_line = tf.keras.preprocessing.sequence.pad_sequences(token_index_targetline,padding='post', maxlen = maxlen)



**ONE HOT REPRESENTATION**

In [70]:
import numpy as np
encoder_input_data = np.zeros(
    (len(padded_source_line), maxlen, top_k+4), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(padded_source_line), maxlen, top_k+4), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(padded_source_line), maxlen, top_k+4), dtype="float32"
)
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)


for i, (source_l, target_l) in enumerate(zip(padded_source_line, padded_target_line)):
    for t in range(len(source_l)):
      encoder_input_data[i, t, source_l[t]] = 1.0

    for t in range(len(target_l)):
      decoder_input_data[i, t, target_l[t]] = 1.0
      if t>0:
        decoder_target_data[i, t-1 , target_l[t]] = 1.0

(14643, 30, 1004)
(14643, 30, 1004)
(14643, 30, 1004)


**Convert to and PAD the index Sequence and One Hot Representation (Valid Set)**

In [71]:
import numpy as np
data_valid=pd.read_csv('/content/gdrive/MyDrive/valid.csv')
source_line_valid=data_valid['sourceLineTokens']
target_line_valid=data_valid['targetLineTokens']

#Convert to index sequence
token_index_sourceline_valid=convert_to_index_seq(source_line_valid)
token_index_targetline_valid=convert_to_index_seq(target_line_valid)

#pad the token_index_seq (Valid Set)
padded_source_line_valid = tf.keras.preprocessing.sequence.pad_sequences(token_index_sourceline_valid,padding='post', maxlen = maxlen)
padded_target_line_valid = tf.keras.preprocessing.sequence.pad_sequences(token_index_targetline_valid,padding='post', maxlen = maxlen)

#One Hot representation
encoder_input_data_valid = np.zeros(
    (len(padded_source_line_valid), maxlen, top_k+4), dtype="float32"
)
decoder_input_data_valid = np.zeros(
    (len(padded_source_line_valid), maxlen, top_k+4), dtype="float32"
)
decoder_target_data_valid = np.zeros(
    (len(padded_source_line_valid), maxlen, top_k+4), dtype="float32"
)


for i, (source_l, target_l) in enumerate(zip(padded_source_line_valid, padded_target_line_valid)):
    for t in range(len(source_l)):
      encoder_input_data_valid[i, t, source_l[t]] = 1.0

    for t in range(len(target_l)):
      decoder_input_data_valid[i, t, target_l[t]] = 1.0
      if t>0:
        decoder_target_data_valid[i, t-1 , target_l[t]] = 1.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


**BUILD THE MODEL**

In [72]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, top_k+4))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_y, state_x = encoder(encoder_inputs)
encoder_states = [state_y, state_x]

# Configure decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None, top_k+4))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(top_k+4, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None, 1004)] 0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None, 1004)] 0                                            
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, 512), (None, 3106816     input_7[0][0]                    
__________________________________________________________________________________________________
lstm_7 (LSTM)                   [(None, None, 512),  3106816     input_8[0][0]                    
                                                                 lstm_6[0][1]              

**COMPILE THE MODEL**

In [73]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

**SAVE CHECK POINTS**

In [74]:
from keras.callbacks import ModelCheckpoint
filepath="/content/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

**FIT THE MODEL**

In [75]:

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([encoder_input_data_valid, decoder_input_data_valid],
    decoder_target_data_valid),callbacks=callbacks_list
)

# Save model
model.save("myModel")

Epoch 1/60
229/229 [==============================] - 14s 45ms/step - loss: 2.1882 - accuracy: 0.6232 - val_loss: 1.2547 - val_accuracy: 0.7449

Epoch 00001: val_accuracy improved from -inf to 0.74487, saving model to /content/weights.best.hdf5
Epoch 2/60
229/229 [==============================] - 9s 41ms/step - loss: 1.1627 - accuracy: 0.7574 - val_loss: 0.8640 - val_accuracy: 0.8068

Epoch 00002: val_accuracy improved from 0.74487 to 0.80679, saving model to /content/weights.best.hdf5
Epoch 3/60
229/229 [==============================] - 9s 41ms/step - loss: 0.8615 - accuracy: 0.8098 - val_loss: 0.7681 - val_accuracy: 0.8215

Epoch 00003: val_accuracy improved from 0.80679 to 0.82152, saving model to /content/weights.best.hdf5
Epoch 4/60
229/229 [==============================] - 9s 41ms/step - loss: 0.7645 - accuracy: 0.8262 - val_loss: 0.7132 - val_accuracy: 0.8329

Epoch 00004: val_accuracy improved from 0.82152 to 0.83291, saving model to /content/weights.best.hdf5
Epoch 5/60
229

INFO:tensorflow:Assets written to: myModel/assets


INFO:tensorflow:Assets written to: myModel/assets


**Download the Model**

In [76]:
!zip -r /content/myModel.zip /content/myModel
from google.colab import files
files.download("/content/myModel.zip")

  adding: content/myModel/ (stored 0%)
  adding: content/myModel/saved_model.pb (deflated 90%)
  adding: content/myModel/variables/ (stored 0%)
  adding: content/myModel/variables/variables.index (deflated 63%)
  adding: content/myModel/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/myModel/assets/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Load the Model**

In [77]:
from tensorflow import keras
model = keras.models.load_model("myModel")

# load weights
#model.load_weights("/content/weights.best.hdf5")

encoder_inputs = model.input[0]
encoder_outputs, state_y_enc, state_x_enc = model.layers[2].output
encoder_states = [state_y_enc, state_x_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_y = Input(shape=(latent_dim,), name="input_9")
decoder_state_input_x = Input(shape=(latent_dim,), name="input_10")
decoder_states_inputs = [decoder_state_input_y, decoder_state_input_x]

decoder_lstm = model.layers[3]
decoder_outputs, state_y_dec, state_x_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_y_dec, state_x_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)



**Model Evaluation**

In [78]:
model.evaluate([encoder_input_data_valid, decoder_input_data_valid],
    decoder_target_data_valid,verbose=2)

81/81 - 2s - loss: 0.5781 - accuracy: 0.9133


[0.578093945980072, 0.9132868647575378]

In [79]:
#-----------------Function for predicting and decoding output------------------------------- 
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, top_k+4))
    target_seq[0, 0, 1] = 1.0
    stop_condition = False
    decoded_Token = []
    while not stop_condition:
        output_tokens, y, x = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_Token = index2token[sampled_token_index]      
        if sampled_Token == 'EOS' or len(decoded_Token) > maxlen:
          stop_condition = True
        else:
          if sampled_Token not in list(['SOS','EOS','PAD']):
            decoded_Token.append(sampled_Token)  
        target_seq = np.zeros((1, 1, top_k+4))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [y, x]
    return decoded_Token

In [80]:
#--------------Token Decoding for test data set-----------------
import nltk
BleuScore=0
for seq_index in range(100):
    # Take one sequence for decoding (part of the training set)
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_Token = decode_sequence(input_seq)
    BleuScore+= nltk.translate.bleu_score.sentence_bleu(vocab.target_line[seq_index], decoded_Token)
    print("-------------------------------------------")
    print(seq_index)
    print("Input sentence  :", vocab.source_line[seq_index])
    print("Target sentence :", vocab.target_line[seq_index])
    print("Decoded sentence:", decoded_Token)
print("BleuScore:",BleuScore/len(vocab.target_line)*100,"%")

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


-------------------------------------------
0
Input sentence  : ['if', '(', 'num', '%', 'i', '!=', '=', '0', ')']
Target sentence : ['if', '(', 'num', '%', 'i', '==', '0', ')']
Decoded sentence: ['if', '(', 'num', '%', 'i', '!=', '0', ')', ';']
-------------------------------------------
1
Input sentence  : []
Target sentence : ['}']
Decoded sentence: ['}']
-------------------------------------------
2
Input sentence  : ['d', '=', '(', 'b', '*', 'b', ')', '-', '4', '*', 'a', '*', 'c']
Target sentence : ['d', '=', '(', 'b', '*', 'b', ')', '-', '4', '*', 'a', '*', 'c', ';']
Decoded sentence: ['d', '=', '(', 'b', '*', 'b', ')', '-', '4', '*', 'a', '*', 'c', ';']
-------------------------------------------
3
Input sentence  : ['printf', '(', '"Let\\\\s \\"', 'C', '" ! "', ')', ';']
Target sentence : ['printf', '(', '"Let\\\\s \\"', 'C', '\\', '"!"', ')', ';']
Decoded sentence: ['printf', '(', '"Let\\\\s \\"', 'C', '\\', '"!"', ')', ';']
-------------------------------------------
4
Input s

In [81]:
#--------------Token Decoding for valid data Set------------------
import nltk
Fixed_Token=[]
BleuScore=0
for seq_index in range(len(source_line_valid)):
    # Take one sequence for decoding (part of the valid set)
    input_seq = encoder_input_data_valid[seq_index : seq_index + 1]
    decoded_Token = decode_sequence(input_seq)
    Fixed_Token.append(decoded_Token)
    BleuScore+= nltk.translate.bleu_score.sentence_bleu(target_line_valid[seq_index], decoded_Token)
    #print("-------------------------------------------")
    print(seq_index)
    #print("Input sentence  :", source_line_valid[seq_index])
    #print("Target sentence :", target_line_valid[seq_index])
    #print("Decoded sentence:", decoded_Token)
print("BleuScore:",BleuScore/len(target_line_valid)*100,"%")

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [82]:
#------------Create the CSV outfile containing FixedToken-------------#
import csv
import pandas as pd
with open('/content/gdrive/MyDrive/valid.csv', 'r') as read_f, \
        open('/content/gdrive/MyDrive/valid_output.csv', 'w', newline='') as write_f:
        csv_reader = csv.reader(read_f)
        csv_writer = csv.writer(write_f)
        i = 0
        for row in csv_reader:
          if(i==0):
            row.append(str("FixedToken"))
          else:
            row.append(Fixed_Token[i-1])          
          csv_writer.writerow(row)
          i += 1
          if(i>len(Fixed_Token)):
            break
print("File 'valid_output.csv' created ")

 

File 'valid_output.csv' created 


In [ ]:
!python Demo_eval.py valid.csv out.csv
